In [1]:
import os

os.chdir('../')
os.getcwd()

'/home/shivansh/repos/assembly_glovebox_dataset'

In [2]:
import torch
import torch.nn.functional as F
import torchmetrics

from training.lightning_trainers.lightning_model import LitModel
from training.lightning_trainers.mobilesam_model import MobileSamLitModel
from training.lightning_trainers.bisenetv2_model import BiSeNetV2Model

from lightning.pytorch import loggers as pl_loggers
from training.dataloaders.datamodule import AssemblyDataModule
import lightning.pytorch as pl

import time
import os

In [3]:
test_dropout = False


def get_path(model_type):
    if model_type == "unet":
        path_1 = "./lightning_logs/version_73/checkpoints/epoch=1011-step=8096.ckpt"
        model_1 = LitModel.load_from_checkpoint(path_1, map_location=torch.device('cuda:0'), test_dropout=test_dropout)

        path_2 = "./lightning_logs/version_76/checkpoints/epoch=1856-step=29712.ckpt"
        model_2 = LitModel.load_from_checkpoint(path_2, map_location=torch.device('cuda:0'), test_dropout=test_dropout)

        path_3 = "./lightning_logs/version_78/checkpoints/epoch=3223-step=51584.ckpt"
        model_3 = LitModel.load_from_checkpoint(path_3, map_location=torch.device('cuda:0'), test_dropout=test_dropout)

    if model_type == "bisenetv2":
        path_1="./lightning_logs/version_79/checkpoints/epoch=3373-step=26992.ckpt"
        model_1 = BiSeNetV2Model.load_from_checkpoint(path_1, map_location=torch.device('cuda:1'), test_dropout=test_dropout)

        
        path_2="./lightning_logs/version_77/checkpoints/epoch=1083-step=8672.ckpt"
        model_2 = BiSeNetV2Model.load_from_checkpoint(path_2, map_location=torch.device('cuda:1'), test_dropout=test_dropout)

        path_3="./lightning_logs/version_72/checkpoints/epoch=1440-step=11528.ckpt"
        model_3 = BiSeNetV2Model.load_from_checkpoint(path_3, map_location=torch.device('cuda:1'), test_dropout=test_dropout)

    if model_type == "mobilesam":
        
        path_1="./lightning_logs/version_80/checkpoints/epoch=3027-step=24224.ckpt"
        model_1 = MobileSamLitModel.load_from_checkpoint(path_1, map_location=torch.device('cuda:2'), test_dropout=test_dropout)
        
        path_2="./lightning_logs/version_75/checkpoints/epoch=1849-step=29600.ckpt"
        model_2 = MobileSamLitModel.load_from_checkpoint(path_2, map_location=torch.device('cuda:2'), test_dropout=test_dropout)

        path_3="./lightning_logs/version_69/checkpoints/epoch=1312-step=10704.ckpt"
        model_3 = MobileSamLitModel.load_from_checkpoint(path_3, map_location=torch.device('cuda:2'), test_dropout=test_dropout)

    return [model_1, model_2, model_3]

# 1. Warm up the gpu

TODO: 
- add dropout
- right image loading (divide by batch across entire set using dataloader)
- 

Edit back CUDA device change in get_prediction in mobilesam_model.py

# 2. Load an image to use from dataset

In [4]:
image_0 = torch.randn(1, 3, 256, 256).cuda(device='cuda:0')
image_1 = torch.randn(2, 3, 256, 256).cuda(device='cuda:1')
image_2 = torch.randn(1, 3, 256, 256).cuda(device='cuda:2')

# 3. Test Time Predictions for Each Model

### unet

In [5]:
models = get_path("unet")

ensemble_time = 0
for model in models:
    start_time = time.time()

    output = model.model(image_0)

    end_time = time.time()

    ensemble_time += end_time - start_time

ensemble_time

0.6510903835296631

In [6]:
start_time = time.time()

model_1 = models[1]

output = model_1.model(image_0)

end_time = time.time()

base_time =  end_time - start_time

base_time

0.0038001537322998047

### bisenet

In [7]:
models = get_path("bisenetv2")

ensemble_time = 0
for model in models:
    start_time = time.time()

    output = model.model(image_1)

    end_time = time.time()

    ensemble_time += end_time - start_time

ensemble_time / image_1.shape[0] # bisenet needs 2 images passed

0.20866870880126953

In [8]:
start_time = time.time()

model_1 = models[0]

output = model_1.model(image_1)

end_time = time.time()

base_time =  end_time - start_time

base_time / image_1.shape[0]

0.004986286163330078

# mobilesam

In [9]:
image_2.device, model.device

(device(type='cuda', index=2), device(type='cuda', index=1))

In [10]:
models = get_path("mobilesam")

ensemble_time = 0
for model in models:
    
    start_time = time.time()

    output = model._get_preds(image_2)

    end_time = time.time()

    ensemble_time += end_time - start_time

ensemble_time

0.38238000869750977

In [13]:
start_time = time.time()

model_1 = models[1].to(image_2.device)

output = model_1._get_preds(image_2)

end_time = time.time()

base_time =  end_time - start_time

base_time / image_1.shape[0]

0.028254389762878418